In [ ]:
# huggingface-cli login
# HF_TOKEN=some_token_here

In [ ]:

from datasets import load_dataset
import json
# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("clinicalnlplab/DDI2013_test")


print(ds)


def processing(dataset, output_path):
    res = []
    for index, item in enumerate(dataset):
        text = item['query'].split("INPUT: ")[-1].split("What is the relationship between")[0].strip()
        answer = item['answer']
        gold = item['gold']

        dict_item = {
            'id': index,
            'sentence': text,
            'answer': answer,
            'label': gold
        }
        res.append(dict_item)

    with open(output_path, 'w') as f:
        json.dump(res, f, indent=4)


processing(ds['train'], 'train.json')
processing(ds['test'], 'test.json')
processing(ds['valid'], 'val.json')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/616 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.88M [00:00<?, ?B/s]

data/valid-00000-of-00001.parquet:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18779 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/7244 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5761 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'query', 'answer', 'choices', 'gold'],
        num_rows: 18779
    })
    valid: Dataset({
        features: ['id', 'query', 'answer', 'choices', 'gold'],
        num_rows: 7244
    })
    test: Dataset({
        features: ['id', 'query', 'answer', 'choices', 'gold'],
        num_rows: 5761
    })
})


In [ ]:
# !pip install -q transformers datasets

In [ ]:
import pandas as pd
import json

def load_json_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return pd.DataFrame(data)

train_df = load_json_data('train.json')
val_df = load_json_data('val.json')
test_df = load_json_data('test.json')

print("Training data shape:", train_df.shape)
print("Validation data shape:", val_df.shape)
print("Test data shape:", test_df.shape)

display(train_df.head())

Training data shape: (18779, 4)
Validation data shape: (7244, 4)
Test data shape: (5761, 4)


,id,sentence,answer,label
0,0,Both efavirenz and nevirapine have been compar...,none,4
1,1,Etonogestrel may interact with the following m...,none,4
2,2,Other concomitant therapy Although specific in...,none,4
3,3,Co-administration of aliskiren did not signifi...,none,4
4,4,No interactions have been observed with beta-r...,none,4


In [ ]:
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(df):
    return tokenizer(
        df['sentence'].tolist(),
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)
test_encodings = tokenize_data(test_df)

train_labels = torch.tensor(train_df['label'].tolist())
val_labels = torch.tensor(val_df['label'].tolist())
test_labels = torch.tensor(test_df['label'].tolist())

print("Training encoding shape:", train_encodings['input_ids'].shape)
print("Validation encoding shape:", val_encodings['input_ids'].shape)
print("Test encoding shape:", test_encodings['input_ids'].shape)

print("Training labels shape:", train_labels.shape)
print("Validation labels shape:", val_labels.shape)
print("Test labels shape:", test_labels.shape)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Training encoding shape: torch.Size([18779, 242])
Validation encoding shape: torch.Size([7244, 274])
Test encoding shape: torch.Size([5761, 173])
Training labels shape: torch.Size([18779])
Validation labels shape: torch.Size([7244])
Test labels shape: torch.Size([5761])


In [ ]:
from transformers import BertForSequenceClassification
import torch

num_labels = len(train_df['label'].unique())
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels).to('cuda')

# Freeze all layers except the classifier
for name, param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

# Verify that only the classifier layer requires gradients
for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert.embeddings.word_embeddings.weight: False
bert.embeddings.position_embeddings.weight: False
bert.embeddings.token_type_embeddings.weight: False
bert.embeddings.LayerNorm.weight: False
bert.embeddings.LayerNorm.bias: False
bert.encoder.layer.0.attention.self.query.weight: False
bert.encoder.layer.0.attention.self.query.bias: False
bert.encoder.layer.0.attention.self.key.weight: False
bert.encoder.layer.0.attention.self.key.bias: False
bert.encoder.layer.0.attention.self.value.weight: False
bert.encoder.layer.0.attention.self.value.bias: False
bert.encoder.layer.0.attention.output.dense.weight: False
bert.encoder.layer.0.attention.output.dense.bias: False
bert.encoder.layer.0.attention.output.LayerNorm.weight: False
bert.encoder.layer.0.attention.output.LayerNorm.bias: False
bert.encoder.layer.0.intermediate.dense.weight: False
bert.encoder.layer.0.intermediate.dense.bias: False
bert.encoder.layer.0.output.dense.weight: False
bert.encoder.layer.0.output.dense.bias: False
bert.encoder

In [ ]:
print(model.device)

cuda:0


In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import numpy as np

class DrugInteractionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DrugInteractionDataset(train_encodings, train_labels)
val_dataset = DrugInteractionDataset(val_encodings, val_labels)
test_dataset = DrugInteractionDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

/tmp/ipython-input-4236155275.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipython-input-4236155275.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss
1,0.626000,0.537333
2,0.563800,0.532659
3,0.584200,0.528191


/tmp/ipython-input-4236155275.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipython-input-4236155275.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
/tmp/ipython-input-4236155275.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipython-input-4236155275.py:14: UserWarning: To copy construct from a tensor, it is recomm

TrainOutput(global_step=882, training_loss=0.5937729806316142, metrics={'train_runtime': 1189.1375, 'train_samples_per_second': 47.376, 'train_steps_per_second': 0.742, 'total_flos': 7006319147778588.0, 'train_loss': 0.5937729806316142, 'epoch': 3.0})

In [ ]:
trainer.evaluate(test_dataset)


/tmp/ipython-input-4236155275.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipython-input-4236155275.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


{'eval_loss': 0.6411791443824768,
 'eval_runtime': 57.0836,
 'eval_samples_per_second': 100.922,
 'eval_steps_per_second': 1.594,
 'epoch': 3.0}

In [ ]:
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import torch

# Get the trained model from the trainer
model = trainer.model
model.eval() # Set the model to evaluation mode
model.to('cuda') # Move the model to GPU if available

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=64)

predictions = []
true_labels = []

# Iterate over the test data and make predictions
for batch in test_dataloader:
    # Move batch to GPU if available
    batch = {k: v.to('cuda') for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1).cpu().numpy()
    labels = batch['labels'].cpu().numpy()

    predictions.extend(preds)
    true_labels.extend(labels)

# Generate and print the classification report
report = classification_report(true_labels, predictions)
print(report)

/tmp/ipython-input-4236155275.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipython-input-4236155275.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       360
           1       0.00      0.00      0.00       302
           2       0.00      0.00      0.00       221
           3       0.00      0.00      0.00        96
           4       0.83      1.00      0.91      4782

    accuracy                           0.83      5761
   macro avg       0.17      0.20      0.18      5761
weighted avg       0.69      0.83      0.75      5761



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
